# 마스크 이미지 증강

증강 프로세스

1. 마스크를 찾용하지 않은 얼굴 이미지: 얼굴 인식 및 얼굴 이미지 추출
2. 마스크를 착용한 이미지 생성: 추출한 얼굴에 마스크 이미지를 추가하여 마스크를 착용한 이미지 생성

In [ ]:
from google.colab import drive
import os
import gc
import shutil
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image

import face_recognition

In [ ]:
without_mask = '/content/drive/MyDrive/mask/combined_without_mask'

without_mask_images = [os.path.join(without_mask, image) for image in os.listdir(without_mask)]

In [ ]:
len(without_mask_images)

In [ ]:
face_split_path = '/content/drive/MyDrive/mask/combined_without_mask_face'
if not os.path.exists(face_split_path):
  os.makedirs(face_split_path)

In [ ]:
for i in range(len(without_mask_images)):
  image_path = without_mask_images[i]

  image = cv2.imread(image_path)

  # 이미지에서 사람 얼굴 bounding boxes 반환
  face_locations = face_recognition.face_locations(image, model = 'hog')

  # 얼굴 인식 여부 확인
  found_face = False
  for face_location in face_locations:
    (y1, x2, y2, x1) = face_location

    found_face = True # 얼굴이 있음

  if found_face:

    y1 = max(0, y1)
    y2 = min(image.shape[0], y2)
    x1 = max(0, x1)
    x2 = min(image.shape[1], x2)

    image = image[y1:y2, x1:x2]

    if image.size == 0:
      print(f'잘린 이미지가 비어있음: {image_path}')
      continue

    image_path, extension = os.path.splitext(image_path)
    image_name = image_path.split('/')[-1]
    without_mask_image_path = os.path.join('/content/drive/MyDrive/mask/combined_without_mask_face', image_name + extension)

    cv2.imwrite(without_mask_image_path, image)

    print('Saved', without_mask_image_path)

  else:
    print('얼굴 인식 못함.', image_path)

In [ ]:
white_mask_image = '/content/drive/MyDrive/mask/white-mask.png' # 흰색 마스크
black_mask_image = '/content/drive/MyDrive/mask/black-mask.png' # 검정색 마스크
blue_mask_image = '/content/drive/MyDrive/mask/blue-mask.png' # 파란색 마스크

In [ ]:
def face_angle_recognition(chin_point, nose_point):

  angle = np.arctan2(chin_point[1] - nose_point[1], chin_point[0] - nose_point[0])

  return angle

def mask_on_face(face_image, face_landmark, mask_image):

  nose_bridge = face_landmark['nose_bridge']
  chin = face_landmark['chin']

  nose_point = nose_bridge[0] # 콧등 상단점
  chin_point = chin[len(chin) // 2] # 턱 중간점

  mask_width = chin[-1][0] - chin[0][0] # 마스크 너비
  mask_height = chin_point[1] - nose_point[1] # 마스크 높이

  angle = face_angle_recognition(chin_point, nose_point) # 얼굴 각도

  mask_image = mask_image.resize((mask_width, mask_height)) # 마스크 크기 조정

  mask_image_rotation = mask_image.rotate(angle, expand=True)

  mask_center_x = (nose_point[0] + chin_point[0]) // 2
  mask_center_y = (nose_point[1] + chin_point[1]) // 2

  # point_x = nose_point[0] - (mask_width // 2)
  # point_y = nose_point[1]

  mask_w, mask_h = mask_image_rotation.size

  point_x = mask_center_x - mask_w // 2
  point_y = mask_center_y - mask_h // 2

  face_image.paste(mask_image_rotation, (point_x, point_y), mask_image_rotation)

  return face_image

def create_mask(image_path, mask_path):

  key_points = ('nose_bridge', 'chin')

  # 얼굴 랜드마크 추출
  load_image = face_recognition.load_image_file(image_path)
  face_locations = face_recognition.face_locations(load_image, model = 'hog')
  face_landmarks = face_recognition.face_landmarks(load_image, face_locations)

  face_image = Image.fromarray(load_image)
  mask_image = Image.open(mask_path)

  found_face = False

  for face_landmark in face_landmarks:

    for point in key_points:

      if point not in face_landmark:
        continue

    found_face = True

    face_image = mask_on_face(face_image, face_landmark, mask_image)

  if found_face:
    image_path, extension = os.path.splitext(image_path)
    image_name = image_path.split('/')[-1]
    print(image_name)
    try:
      with_mask_image_path = os.path.join('/content/drive/MyDrive/mask/combined_with_mask_face', image_name + extension)
    except:
      image_name = 'without_mask_' + image_name
      with_mask_image_path = os.path.join('/content/drive/MyDrive/mask/combined_with_mask_face', image_name + extension)

    # face_image = np.array(face_image)
    # cv2.imwrite(with_mask_image_path, face_image)

    face_image.save(with_mask_image_path)

  else:
    print(f'얼굴 인식 못함. {image_path}')

In [ ]:
without_mask_face = '/content/drive/MyDrive/mask/combined_without_mask_face'
without_mask_face_images = [os.path.join(without_mask_face, image) for image in os.listdir(without_mask_face)]
len(without_mask_face_images)

In [ ]:
face_split_path = '/content/drive/MyDrive/mask/combined_with_mask_face'
if not os.path.exists(face_split_path):
  os.makedirs(face_split_path)
else:
  print('exist.')

In [ ]:
for i in range(len(without_mask_face_images)):
  if i < 796:
    create_mask(without_mask_face_images[i], white_mask_image)

  elif i < 1592:
    create_mask(without_mask_face_images[i], black_mask_image)

  else:
    create_mask(without_mask_face_images[i], blue_mask_image)

In [ ]:
def face_angle_recognition(chin_point, upper_lip_point):

  angle = np.arctan2(chin_point[1] - upper_lip_point[1], chin_point[0] - upper_lip_point[0])

  return angle

def imp_mask_on_face(face_image, face_landmark, mask_image):

  mouth_top = face_landmark['top_lip']
  chin = face_landmark['chin']

  mouth_top_point = mouth_top[0] # 입술 상단점
  chin_point = chin[len(chin) // 2] # 턱 중간점

  mask_width = chin[-1][0] - chin[0][0] # 마스크 너비
  mask_height = chin_point[1] - mouth_top_point[1] # 마스크 높이

  angle = face_angle_recognition(chin_point, mouth_top_point) # 얼굴 각도

  mask_image = mask_image.resize((mask_width, mask_height)) # 마스크 크기 조정

  mask_image_rotation = mask_image.rotate(angle, expand=True)

  mask_center_x = (mouth_top_point[0] + chin_point[0]) // 2
  mask_center_y = (mouth_top_point[1] + chin_point[1]) // 2

  # point_x = mask_center_x - (mask_width // 2)
  # point_y = mouth_top_point[1]

  mask_w, mask_h = mask_image_rotation.size

  point_x = mask_center_x - mask_w // 2
  point_y = mask_center_y - mask_h // 2

  face_image.paste(mask_image_rotation, (point_x, point_y), mask_image_rotation)

  return face_image

def create_imp_mask(image_path, mask_path):

  key_points = ('nose_bridge', 'chin')

  # 얼굴 랜드마크 추출
  load_image = face_recognition.load_image_file(image_path)
  face_locations = face_recognition.face_locations(load_image, model = 'hog')
  face_landmarks = face_recognition.face_landmarks(load_image, face_locations)

  face_image = Image.fromarray(load_image)
  mask_image = Image.open(mask_path)

  found_face = False

  for face_landmark in face_landmarks:

    for point in key_points:

      if point not in face_landmark:
        continue

    found_face = True

    face_image = imp_mask_on_face(face_image, face_landmark, mask_image)

  if found_face:
    image_path, extension = os.path.splitext(image_path)
    image_name = image_path.split('/')[-1]
    print(image_name)
    try:
      with_mask_image_path = os.path.join('/content/drive/MyDrive/mask/combined_half_mask_face', image_name + extension)
    except:
      image_name = 'without_mask_' + image_name
      with_mask_image_path = os.path.join('/content/drive/MyDrive/mask/combined_half_mask_face', image_name + extension)

    # face_image = np.array(face_image)
    # cv2.imwrite(with_mask_image_path, face_image)

    face_image.save(with_mask_image_path)

  else:
    print(f'얼굴 인식 못함. {image_path}')

In [ ]:
without_mask_face = '/content/drive/MyDrive/mask/combined_without_mask_face'
without_mask_face_images = [os.path.join(without_mask_face, image) for image in os.listdir(without_mask_face)]
len(without_mask_face_images)

In [ ]:
face_split_path = '/content/drive/MyDrive/mask/combined_half_mask_face'
if not os.path.exists(face_split_path):
  os.makedirs(face_split_path)
else:
  print('exist.')

In [ ]:
for i in range(len(without_mask_face_images)):
  if i < 796:
    create_imp_mask(without_mask_face_images[i], white_mask_image)

  elif i < 1592:
    create_imp_mask(without_mask_face_images[i], black_mask_image)

  else:
    create_imp_mask(without_mask_face_images[i], blue_mask_image)

# 증강 이미지 병합

In [ ]:
source_folder_mask = '/content/drive/MyDrive/mask/combined_with_mask'
source_folder_face_mask = '/content/drive/MyDrive/mask/combined_with_mask_face'

for filename in os.listdir(source_folder_face_mask):
  source_file = os.path.join(source_folder_face_mask, filename)

  filename, file_extension = os.path.splitext(filename)

  aug_filename = f'{filename}_aug{file_extension}'
  # aug_filename = filename.split('.')[0] + '_aug' + '.' + filename.split('.')[1]

  target_file = os.path.join(source_folder_mask, aug_filename)

  shutil.copy(source_file, target_file)

In [ ]:
with_mask = '/content/drive/MyDrive/mask/combined_with_mask'
with_mask_images = [os.path.join(with_mask, image) for image in os.listdir(with_mask) if '_aug' in image]
len(with_mask_images)

In [ ]:
with_mask = '/content/drive/MyDrive/mask/combined_with_mask'
with_mask_images = [os.path.join(with_mask, image) for image in os.listdir(with_mask)]
len(with_mask_images)

In [ ]:
source_folder_without_mask = '/content/drive/MyDrive/mask/combined_without_mask'
source_folder_without_face_mask = '/content/drive/MyDrive/mask/combined_without_mask_face'

for filename in os.listdir(source_folder_without_face_mask):
  source_file = os.path.join(source_folder_without_face_mask, filename)

  filename, file_extension = os.path.splitext(filename)

  aug_filename = f'{filename}_aug{file_extension}'
  # aug_filename = filename.split('.')[0] + '_aug' + '.' + filename.split('.')[1]

  target_file = os.path.join(source_folder_without_mask, aug_filename)

  shutil.copy(source_file, target_file)

In [ ]:
without_mask = '/content/drive/MyDrive/mask/combined_without_mask'
without_mask_images = [os.path.join(without_mask, image) for image in os.listdir(without_mask) if '_aug' in image]
len(without_mask_images)

In [ ]:
without_mask = '/content/drive/MyDrive/mask/combined_without_mask'
without_mask_images = [os.path.join(without_mask, image) for image in os.listdir(without_mask)]
len(without_mask_images)

In [ ]:
source_folder_mask = '/content/drive/MyDrive/mask/combined_half_mask'
source_folder_face_mask = '/content/drive/MyDrive/mask/combined_half_mask_face'

for filename in os.listdir(source_folder_face_mask):
  source_file = os.path.join(source_folder_face_mask, filename)

  aug_filename = filename.split('.')[0] + '_aug' + '.' + filename.split('.')[1]
  target_file = os.path.join(source_folder_mask, aug_filename)

  shutil.copy(source_file, target_file)

In [ ]:
without_mask = '/content/drive/MyDrive/mask/combined_half_mask'
without_mask_images = [os.path.join(without_mask, image) for image in os.listdir(without_mask) if '_aug' in image]
len(without_mask_images)

In [ ]:
without_mask = '/content/drive/MyDrive/mask/combined_half_mask'
without_mask_images = [os.path.join(without_mask, image) for image in os.listdir(without_mask)]
len(without_mask_images)

In [ ]:
# 부적절한 마스크 착용 + 마스크 미착용 병합
source_folder_without_mask = '/content/drive/MyDrive/mask/combined_without_mask'
combined_half_mask = '/content/drive/MyDrive/mask/combined_half_mask'

for filename in os.listdir(combined_half_mask):
  source_file = os.path.join(combined_half_mask, filename)

  filename, file_extension = os.path.splitext(filename)

  imp_filename = f'{filename}_imp{file_extension}'
  # imp_filename = filename.split('.')[0] + '_imp' + '.' + filename.split('.')[1]

  target_file = os.path.join(source_folder_without_mask, imp_filename)

  shutil.copy(source_file, target_file)